In [2]:
import numpy as np
from scipy import stats


In [3]:
#读数据，处理数据
data=np.loadtxt(open('x.csv','rb'),delimiter=',')
print(data)
print(data[10,:])
print(data[0:3,:])

[[-0.772081  3.30056  -2.14486 ]
 [-1.54372   1.91863  -7.25592 ]
 [ 6.61561  -1.37613   6.9167  ]
 ...
 [-6.03685   1.94148  -5.27942 ]
 [ 3.11438  -2.1821    1.64958 ]
 [ 3.87056  -2.64687   3.22179 ]]
[ 2.28528 -1.97768  3.10797]
[[-0.772081  3.30056  -2.14486 ]
 [-1.54372   1.91863  -7.25592 ]
 [ 6.61561  -1.37613   6.9167  ]]


In [4]:
#定义GMM类


class gmm():
    def __init__(self,k,mu,sigma,samp):
        self.k=k
        self.samp=samp
        self.alpha=np.ones([k])/k
        self.mean=mu#均值应该是一个矩阵，每一行为一个成分的均值初始值
        self.var=sigma#协方差矩阵是一个张量，每一张是一个成分的协方差初始值
        
    def post(self):
        #为每个样本计算后验
        num_samp=self.samp.shape[0]
        post_mat=np.zeros([num_samp,self.k])
        temp_mat=np.zeros([num_samp,self.k])
        for i_k in range(self.k):
            dist_nor=stats.multivariate_normal(self.mean[i_k,:],self.var[i_k,:,:])
            #temp_result=dist_nor.pdf(self.samp[:,:]).reshape(1,-1).T
            #print(temp_mat)
            temp_mat[:,i_k]=dist_nor.pdf(self.samp[:,:])
        alpha_mat=np.diag(self.alpha)
        temp_mat=np.dot(temp_mat,alpha_mat)
        sum_post=temp_mat.sum(axis=1).reshape(1,-1).T
        post_mat=temp_mat/sum_post
        print(np.log(sum_post).sum())
        return post_mat
    
    def update_paras(self,post_mat):
        num_samp=self.samp.shape[0]
        dim=self.samp.shape[1]#样本维度
        alpha=np.zeros([self.k])
        mean=np.zeros([self.k,dim])
        sigma=np.zeros([self.k,dim,dim])
        for i_k in range(self.k):
            weights=np.diag(post_mat[:,i_k])
            temp_mu=np.dot(weights,self.samp).sum(axis=0)
            temp_sum=post_mat[:,i_k].sum()
            mean[i_k,:]=temp_mu/temp_sum
        
        for i_k in range(self.k):
            weights=np.diag(post_mat[:,i_k])
            temp_sigma=np.dot(np.dot((self.samp-mean[i_k,:]).T,weights),self.samp-mean[i_k,:])
            temp_sum=post_mat[:,i_k].sum()
            sigma[i_k,:,:]=temp_sigma/temp_sum
            
        for i_k in range(self.k):
            temp_sum=post_mat[:,i_k].sum()
            alpha[i_k]=temp_sum/num_samp
            
        self.alpha=alpha
        self.mean=mean
        self.var=sigma
        
        return mean,sigma,alpha
    
            
            
            
        
            
            

In [6]:
#初始化
k=5
train_time=26
mean=data[0:k,:]#specify initial mean value of gmm
print('initial mean',mean)
sigma=np.zeros([k,data.shape[1],data.shape[1]])
std=np.array([[1,0,0],[0,1,0],[0,0,1]])
sigma[:,:,:]=std
print('initial sigma',sigma)
gmm_1=gmm(k,mean,sigma,data)
for iters in range(train_time):
    print('the likelihood of samples at '+str(iters+1)+' is')
    post_mat=gmm_1.post()
    #print(post_mat)
    mean_r,sigma_r,alpha_r=gmm_1.update_paras(post_mat)

#print(mean_r,sigma_r,alpha_r)
print('mean_r')
print(mean_r)
print('sigma')
print(sigma_r)
print('alpha')
print(alpha_r)
print(alpha_r.sum())
print('z value ',post_mat)
np.savetxt('z.csv',post_mat,delimiter=',')
np.savetxt('params1.dat',mean_r,delimiter=',')
#out_sigma=np.vstack([sigma_r[0,:,:],sigma_r[1,:,:],sigma_r[2,:,:]])
#np.savetxt('params2.dat',out_sigma,delimiter=',')
np.savetxt('params3.dat',alpha_r,delimiter=',')

initial mean [[-0.772081  3.30056  -2.14486 ]
 [-1.54372   1.91863  -7.25592 ]
 [ 6.61561  -1.37613   6.9167  ]
 [-5.51916   2.18445  -2.91299 ]
 [ 4.7442   -1.61953   4.61999 ]]
initial sigma [[[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]]
the likelihood of samples at 1 is
-165576.2185663919
the likelihood of samples at 2 is
-62126.465446494
the likelihood of samples at 3 is
-60656.828185134524
the likelihood of samples at 4 is
-60348.28421330417
the likelihood of samples at 5 is
-60288.297511227014
the likelihood of samples at 6 is
-60252.70920585645
the likelihood of samples at 7 is
-60216.816247957344
the likelihood of samples at 8 is
-60157.887202337355
the likelihood of samples at 9 is
-60013.94920511053
the likelihood of samples at 10 is
-59578.270679698755
the likelihood of samples at 11 is
-58708.36019914955
the likelihood